In [25]:
import time
from functools import wraps
def timethis(func):
    @wraps(func)  # @wraps会复制被装饰器函数元信息给装饰器
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

@timethis
def countdown(n: int) -> None:
    '''Counts down'''
    while n > 0:
        n -= 1
        
countdown(10**6)
print(countdown.__name__)
print(countdown.__doc__)
print(countdown.__annotations__)
print(countdown.__wrapped__) # @wraps能让你通过属性__wrapped__直接访问被装饰函数
from inspect import signature
print(signature(countdown)) # __wrapped__能让被装饰函数正确暴露底层的参数签名信息

countdown 0.0468747615814209
countdown
Counts down
{'n': <class 'int'>, 'return': None}
<function countdown at 0x000001DF551FA3A8>


In [24]:
import time
from functools import wraps
def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

@timethis
def countdown(n: int) -> None:
    '''Counts down'''
    while n > 0:
        n -= 1
        
countdown(10**6)
# 没有使用@wraps则被装饰器函数countdown丢失所有元信息
print(countdown.__name__)
print(countdown.__doc__)
print(countdown.__annotations__)

countdown 0.045876502990722656
wrapper
None
{}


In [35]:
from inspect import signature
from functools import wraps

# 利用装饰器强制函数上的类型检查
def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking
        if not __debug__:
            return func

        # Map function argument names to supplied types
        sig = signature(func)
        print(sig)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        print(bound_types)

        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError(
                            'Argument {} must be {}'.format(name, bound_types[name])
                            )
            return func(*args, **kwargs)
        return wrapper
    return decorate

@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)

spam(1, 2, 3)

(x, y, z=42)
OrderedDict([('x', <class 'int'>), ('z', <class 'int'>)])
1 2 3


In [45]:
class C:
    def __init__(self):
          self.__x=None
    def getx(self):
          return self.__x
    def setx(self,value):
          self.__x=value
    def delx(self):
          del self.__x
    x=property(getx,setx,delx,'')
c = C()
c.x = 2
c.x
del c.x

In [53]:
class C(object):
    def __init__(self):
        self._x = None

    @property    
    def x(self):
        return self._x
    
    @x.setter
    def x(self, value):
        self._x = value

    @x.deleter
    def x(self):
        del self._x

c = C()
c.x = 3
print(c.x)

3


In [44]:
class C(object):
    def __init__(self):
        self._x = None

    x = property()

    @x.getter
    def x(self):
        return self._x

    @x.setter
    def x(self, value):
        self._x = value

    @x.deleter
    def x(self):
        del self._x

c = C()
c.x = 2
print(c.x)

2


In [50]:
from functools import wraps
 
class logit(object):
    def __init__(self, logfile='out.log'):
        self.logfile = logfile
 
    def __call__(self, func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # 打开logfile并写入
            with open(self.logfile, 'a') as opened_file:
                # 现在将日志打到指定的文件
                opened_file.write(log_string + '\n')
            # 现在，发送一个通知
            self.notify()
            return func(*args, **kwargs)
        return wrapped_function
 
    def notify(self):
        # logit只打日志，不做别的
        pass

kk = logit() 
print(kk)
def test():
    pass
cc = kk(test)
cc

<function __main__.test()>

In [96]:
from functools import wraps

class A:
    # Decorator as an instance method
    def decorator1(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 1')
            return func(*args, **kwargs)
        return wrapper

    # Decorator as a class method
    @classmethod
    def decorator2(cls, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 2')
            print(args[0].a)
            return func(*args, **kwargs)
        return wrapper

class B(A):
    a = 5
    @A.decorator2
    def bar(self):
        return 2
b = B()
print(b.bar())

Decorator 2
5
2


In [116]:
from functools import wraps
import inspect

def optional_debug(func):
    if 'debug' in inspect.signature(func).parameters:
        raise TypeError('debug argument already defined')

    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)

    sig = inspect.signature(func)
    print(sig.parameters)
    parms = list(sig.parameters.values())
    print(parms)
    parms.append(inspect.Parameter('debug',
                inspect.Parameter.KEYWORD_ONLY,
                default=False))
    print(parms)
    wrapper.__signature__ = sig.replace(parameters=parms)
    return wrapper

@optional_debug
def add(x, y):
    return x+y

print(inspect.signature(add))


OrderedDict([('x', <Parameter "x">), ('y', <Parameter "y">)])
[<Parameter "x">, <Parameter "y">]
[<Parameter "x">, <Parameter "y">, <Parameter "debug=False">]
(x, y, *, debug=False)


In [119]:
def log_getattribute(cls):
    # Get the original implementation
    orig_getattribute = cls.__getattribute__

    # Make a new definition
    def new_getattribute(self, name):
        print('getting:', name)
        return orig_getattribute(self, name)

    # Attach to the class and return
    cls.__getattribute__ = new_getattribute
    return cls

# Example use
@log_getattribute
class A:
    def __init__(self,x):
        self.x = x
    def spam(self):
        pass

a = A(2)
print(a.x)
print(a.spam())

getting: x
2
getting: spam
None


In [125]:
# 创建不能实例化对象的类
class NoInstances(type):
    pass
    def __call__(self, *args, **kwargs):
        raise TypeError("Can't instantiate directly")

# Example
class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')

In [163]:
# 创建单例
class Singleton(type):
    def __init__(self, *args, **kwargs): # 被父元类的__call__方法调用，初始化创建的类对象Spam
        self.__instance = None # 类对象的属性__instance设置为None
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **kwargs): # 隐式被类Spam调用，创建Spam的实例对象
        if self.__instance is None:
            self.__instance = super().__call__(*args, **kwargs) # 调用类的超类的__call__方法，返回类的实例对象
            return self.__instance
        else:
            return self.__instance

# Example
class Spam(metaclass=Singleton):
    def __init__(self):
        print('Creating Spam')

cc = Spam()
cc2 = Spam()
cc == cc2

Creating Spam


True

In [168]:
import weakref

class Cache(object):
    def __init__(self):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary() # 创建一个弱引用值的映射:值删除后,弱引用隐射对应的元素项也会被垃圾回收删除
    
    def __call__(self, *args):
        if args in self.__cache:
            return sefl.__cache[args]
        else:
            obj = super.__call__(*args)
            self.__cache[args] = obj
            return obj
a = Spam('GG')
b = Spam('GG')
a == b # 显示为True

True

In [209]:
from collections import OrderedDict


# A set of descriptors for various types
class Typed:
    '''数据描述器'''
    _expected_type = type(None)
    def __init__(self, name=None):
        self._name = name

    def __set__(self, instance, value): # 访问描述器对象设置属性时，隐式调用该方法_
        if not isinstance(value, self._expected_type):
            raise TypeError('Expected ' + str(self._expected_type))
        instance.__dict__[self._name] = value # 设置实例的__dict__命名空间

class Integer(Typed):
    _expected_type = int

class Float(Typed):
    _expected_type = float

class String(Typed):
    _expected_type = str

# Metaclass that uses an OrderedDict for class body
class OrderedMeta(type):
    ''' 捕获类的定义信息'''
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict) # clsdict即空间钩子__prepare__返回的OrderedDict()，也是类的namespace
        order = []
        for name, value in clsdict.items(): # 找出类属性中所有Typed的实例组成dict，放入类的'_order’属性中
            if isinstance(value, Typed):
                value._name = name # 修改类型对象的_name属性值为属性名，实例属性名和类属性名相同时，只会访问实例属性，类的属性会被隐藏
                order.append(name)
        d['_order'] = order
        return type.__new__(cls, clsname, bases, d)

    @classmethod
    def __prepare__(cls, clsname, bases): # 创建namespace:类属性字典clsdict，返回类型为OrderdDict
        return OrderedDict()

class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        return ','.join(str(getattr(self,name)) for name in self._order)

# Example use
class Stock(Structure):
    name = String() # 创建规定的类对象，实例化时必须按按照类型顺序传入构造函数__init__
    shares = Integer()
    price = Float()

    def __init__(self, name, shares, price): # 要求传入数据的类型顺序为String、Interger、Float，否者属性访问时就会抛出错误
        self.name = name
        self.shares = shares
        self.price = price
s = Stock('good', 100, 490.1)
s.as_csv()

'good,100,490.1'

In [222]:
from collections import OrderedDict

class NoDupOrderedDict(OrderedDict):
    '''自己构造的类字典对象，重复定义则抛出异常'''
    def __init__(self, clsname):
        self.clsname = clsname
        super().__init__()
        
    def __setitem__(self, name, value): # 隐式设置dict时(即将要创建的类的类属性放入该dict种)，name()已key经定义则抛出异常
        if name in self:  
            raise TypeError(f'{name} already defined in {self.clsname}')
        super().__setitem__(name, value)

class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        print(clsdict)
        d['_order'] = [name for name in clsdict if name[0] != '_'] # 取出非'_'开头的key组成一个字典元素
        return type.__new__(cls, clsname, bases, d)
    
    @classmethod
    def __prepare__(cls, clsname, bases): # 在__new__方法之前执行，创建namespace，就是字典类对象
        return NoDupOrderedDict(clsname)

class A(metaclass=OrderedMeta):
    def test(self):
        pass
    def test2(self):
        pass

NoDupOrderedDict([('__module__', '__main__'), ('__qualname__', 'A'), ('test', <function A.test at 0x000001DF55727168>), ('test2', <function A.test2 at 0x000001DF55727798>)])


In [229]:
def test():
    a = 5
    print(a)
test.__dict__

{}